# PGChatMessageHistory

`PGChatMessageHistory` is a an implementation of the the LangChain ChatMessageHistory abstraction using `postgres` as the backend.


## Install

Install the `langchain-postgres` package.

In [ ]:
%pip install --upgrade --quiet langchain-postgres

## Create an engine

The first step is to create a `PGEngine` instance, which does the following:

1. Allows you to create tables for storing documents and embeddings.
2. Maintains a connection pool that manages connections to the database. This allows sharing of the connection pool and helps to reduce latency for database calls.

In [ ]:
from langchain_postgres import PGEngine

# See docker command above to launch a Postgres instance with pgvector enabled.
# Replace these values with your own configuration.
POSTGRES_USER = "langchain"
POSTGRES_PASSWORD = "langchain"
POSTGRES_HOST = "localhost"
POSTGRES_PORT = "6024"
POSTGRES_DB = "langchain"

CONNECTION_STRING = (
    f"postgresql+asyncpg://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}"
    f":{POSTGRES_PORT}/{POSTGRES_DB}"
)

pg_engine = PGEngine.from_connection_string(url=CONNECTION_STRING)

To use psycopg3 driver, set your connection string to `postgresql+psycopg://`

### Initialize a table
The `PGChatMessageHistory` class requires a database table with a specific schema in order to store the chat message history.

The `PGEngine` engine has a helper method `init_chat_history_table()` that can be used to create a table with the proper schema for you.

In [ ]:
TABLE_NAME = "chat history"

pg_engine.init_chat_history_table(table_name=TABLE_NAME)

#### Optional Tip: 💡
You can also specify a schema name by passing `schema_name` wherever you pass `table_name`. Eg:

```python
SCHEMA_NAME="my_schema"

engine.init_chat_history_table(
    table_name=TABLE_NAME,
    schema_name=SCHEMA_NAME     # Default: "public"
)
```

### PGChatMessageHistory

To initialize the `PGChatMessageHistory` class you need to provide only 3 things:

1. `engine` - An instance of a `PGEngine` engine.
1. `session_id` - A unique identifier string that specifies an id for the session.
1. `table_name` : The name of the table within the PG database to store the chat message history.
1. `schema_name` : The name of the database schema containing the chat message history table.

In [ ]:
from langchain_postgres import PGChatMessageHistory

history = PGChatMessageHistory.create_sync(
    pg_engine,
    session_id="test_session",
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
)
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [ ]:
history.messages

#### Cleaning up
When the history of a specific session is obsolete and can be deleted, it can be done the following way.

**Note:** Once deleted, the data is no longer stored in Postgres and is gone forever.

In [ ]:
history.clear()

## 🔗 Chaining

We can easily combine this message history class with [LCEL Runnables](/docs/expression_language/how_to/message_history)

To do this we will use one of [Google's Vertex AI chat models](https://python.langchain.com/docs/integrations/chat/google_vertex_ai_palm)


In [ ]:
# enable Vertex AI API
!gcloud services enable aiplatform.googleapis.com

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_google_vertexai import ChatVertexAI

In [ ]:
GOOGLE_CLOUD_PROJECT_ID = ""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatVertexAI(
    project=GOOGLE_CLOUD_PROJECT_ID, model_name="gemini-2.0-flash-exp"
)

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: PGChatMessageHistory.create_sync(
        pg_engine,
        session_id=session_id,
        table_name=TABLE_NAME,
        # schema_name=SCHEMA_NAME,
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
# This is where we configure the session id
config = {"configurable": {"session_id": "test_session"}}

In [ ]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

In [ ]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)